# ReLU : Rectified Linear Unit

* Sigmoid가 deep한 모델에서 제대로 작동하지 않기 때문에 고안된 방법(미분값이 0에 가까워 지기 때문에 input 값의 영향력이 미미해진다)
* tf.nn.relu(tf.matmul(W,X)+b)
* 단, 마지막 layer에서는 sigmoid 사용 (0~1의 출력을 위해서)
* 이외에도 Leaky ReLU, ELU, Maxout 등이 있다

## Weight intialize

* 초기값을 어떤것을 주느냐에 따라서 달라진다
* RBM이 첫 번째 방법 -> 어려움
* Xavier/he initialization : W = np.random.randn(fan_in, fan_out)/np.sqrt(fan_in/2)
* 강력하고 간단한 방법이다
* 아직도 많은 연구가 진행되고 있는 부분 


    
## Dropout

* 모델이 deep해질수록 overfitting될 가능성이 많아짐
* 랜덤하게 몇가지 뉴런을 무시하는 방법(일반적으로 0.5) -> 평가시엔는 모든 뉴런 사용



In [1]:
# import
import tensorflow as tf
import random

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
# weights & bias for nn layers
W1 = tf.get_variable("W1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer()) # Xavier
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob) # dropout

W2 = tf.get_variable("W2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Optimizer

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7} # 0.7
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

W0806 17:01:39.671809 22104 deprecation.py:323] From <ipython-input-1-ef8160b2c15b>:8: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 17:01:39.676795 22104 deprecation.py:323] From c:\users\tjrdn\miniconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 17:01:39.679789 22104 deprecation.py:323] From c:\users\tjrdn\miniconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Plea

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0806 17:01:40.128590 22104 deprecation.py:323] From c:\users\tjrdn\miniconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0806 17:01:40.133576 22104 deprecation.py:323] From c:\users\tjrdn\miniconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0806 17:01:40.228323 22104 deprecation.py:323] From c:\users\tjrdn\miniconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future v

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W0806 17:01:40.692456 22104 deprecation.py:506] From <ipython-input-1-ef8160b2c15b>:24: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0806 17:01:40.780220 22104 deprecation.py:323] From <ipython-input-1-ef8160b2c15b>:39: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch: 0001 cost = 0.447511739
Epoch: 0002 cost = 0.182949855
Epoch: 0003 cost = 0.138233964
Epoch: 0004 cost = 0.110325795
Epoch: 0005 cost = 0.093954430
Epoch: 0006 cost = 0.083877258
Epoch: 0007 cost = 0.076746362
Epoch: 0008 cost = 0.068060165
Epoch: 0009 cost = 0.060861481
Epoch: 0010 cost = 0.060410527
Epoch: 0011 cost = 0.052689915
Epoch: 0012 cost = 0.049838820
Epoch: 0013 cost = 0.047060093
Epoch: 0014 cost = 0.046555887
Epoch: 0015 cost = 0.040237715
Learning Finished!
Accuracy: 0.9795
